In [1]:
import matplotlib.pyplot as plt
from mxnet import gluon, init, nd, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms

In [2]:
def get_dataloader(batch_size, num_workers):
    transformer = transforms.Compose([
        transforms.ToTensor()
    ])
    train = gluon.data.vision.datasets.FashionMNIST(train=True)
    train = train.transform_first(transformer)
    test = gluon.data.vision.datasets.FashionMNIST(train=False)
    test = test.transform_first(transformer)
    train_iter = gluon.data.DataLoader(train, batch_size, shuffle=True, num_workers=num_workers)
    test_iter = gluon.data.DataLoader(test, batch_size, shuffle=False, num_workers=num_workers)
    return train_iter, test_iter

In [3]:
def softmax(X):
    X_exp = X.exp()
    normalization_constant = X_exp.sum(axis=1, keepdims=True)
    return X_exp / normalization_constant

In [4]:
def evaluate_acc(net, data_iter):
    accumulator = 0
    size = 0
    for X, y in data_iter:
        y_hat = net(X)
        y_hat = softmax(y_hat)
        accumulator += (y_hat.argmax(axis=1)==y.astype('float32')).sum()
        size += len(y)
    return accumulator / size

In [5]:
batch_size = 256
num_workers = 4
train_iter, test_iter = get_dataloader(batch_size, num_workers)

In [6]:
net = nn.Sequential()
net.add(nn.Dense(10))
net.initialize(init.Normal(0.01))

In [7]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [9]:
epochs = 10
for epoch in range(epochs):
    for X, y in train_iter:
        with autograd.record():
            y_hat = net(X)
            l = loss(y_hat, y)
        l.backward()
        trainer.step(batch_size)
    epoch_acc = evaluate_acc(net, train_iter)
    print("Epoch %d, acc: %f" % (epoch, epoch_acc.asscalar()))

Epoch 0, acc: 0.841133
Epoch 1, acc: 0.844767
Epoch 2, acc: 0.844800
Epoch 3, acc: 0.847500
Epoch 4, acc: 0.849050
Epoch 5, acc: 0.849267
Epoch 6, acc: 0.848650
Epoch 7, acc: 0.849333
Epoch 8, acc: 0.852783
Epoch 9, acc: 0.854133


In [6]:
for X, y in train_iter:
    print(X.shape)
    print(y.shape)
    break

(10, 1, 28, 28)
(10,)
